<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github//jayralencar/pictoBERT/blob/main/N-gram models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jayralencar/pictoBERT/blob/main/N-gram models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# N-gram models

This notebook presents the procedure for creating the n-gram models to compare with PictoBERT.

In [ ]:
!wget http://jayr.clubedosgeeks.com.br/pictobert/train_childes_all.pt
!wget http://jayr.clubedosgeeks.com.br/pictobert/test_childes_all.pt
!wget http://jayr.clubedosgeeks.com.br/pictobert/val_childes_all.pt

In [ ]:
!wget http://jayr.clubedosgeeks.com.br/pictobert/childes_all_new.json

/root/capsule/files
Downloading...
From: https://drive.google.com/uc?id=1-2g-GCxjBwESqDn3JByAJABU9Dkuqy0m
To: /root/capsule/files/childes_all_new.json
100%|████████████████████████████████████████| 332k/332k [00:00<00:00, 8.90MB/s]
/root/capsule


In [ ]:
TRAIN_DATASET_PATH = "./train_childes_all.pt"
VAL_DATASET_PATH = "./val_childes_all.pt"
TEST_DATASET_PATH = "./test_childes_all.pt"

TOKENIZER_PATH = "./childes_all_new.json"

In [ ]:
!pip install nltk==3.6.2

     |████████████████████████████████| 1.5MB 29.4MB/s eta 0:00:01
  Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import PreTrainedTokenizerFast

loaded_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)
loaded_tokenizer.pad_token = "[PAD]"
loaded_tokenizer.sep_token = "[SEP]"
loaded_tokenizer.mask_token = "[MASK]"
loaded_tokenizer.cls_token = "[CLS]"
loaded_tokenizer.unk_token = "[UNK]"

In [ ]:
import pickle

train_dataset = pickle.load(open(TRAIN_DATASET_PATH,'rb'))
val_dataset = pickle.load(open(VAL_DATASET_PATH,'rb'))
test_dataset = pickle.load(open(TEST_DATASET_PATH,'rb'))

In [ ]:
train = [loaded_tokenizer.convert_ids_to_tokens(s,skip_special_tokens=True) for s in train_dataset['input_ids'] + val_dataset['input_ids']]
test = [loaded_tokenizer.convert_ids_to_tokens(s,skip_special_tokens=True) for s in test_dataset['input_ids']]

In [ ]:
from nltk.lm import Vocabulary
from nltk.lm.preprocessing import flatten
all = list(flatten(train+test))
vocab = Vocabulary(all)

In [ ]:
from nltk import ngrams
from nltk.lm import MLE
from nltk.lm import Laplace
import numpy as np
import math

In [ ]:
%mkdir ngram-models

SyntaxError: invalid syntax (95185168.py, line 3)

In [ ]:
max_order = 10
entropies = []
ppls = []
for order in range(2,11):
  train_data = [list(ngrams(sentence,order,pad_left=True,left_pad_symbol="[PAD]")) for sentence in train]
  test_data = [list(ngrams(sentence,order,pad_left=True,left_pad_symbol="[PAD]")) for sentence in test]

  lm = MLE(order)

  # lm.fit(train_data, vocabulary_text=list(set(all)))

  lm.fit(train_data, vocabulary_text=list(set(all)))
  lm.fit(train_data)

  with open("./ngram-models/{0}-gram_model.pk".format(order),'wb') as fout:
    pickle.dump(lm, fout)

  # all_scores = [lm.logscore(a[-1],a[:-1]) if lm.logscore(a[-1],a[:-1]) != -math.inf else 0.0 for a in flatten(test_data)]
  # entropy = -1 * np.mean(all_scores)
  list_ = [lm.entropy(test) for test in test_data if lm.entropy(test) != math.inf]
  entropy = np.array(list_).mean()
  entropies.append(entropy)
  ppls.append(math.exp(entropy))
  print("Order:", order)
  print("Cross Entropy:", entropy)
  print("Perplexity:", math.exp(entropy))
  print()